<a href="https://colab.research.google.com/github/Rishabhc711/Federated_in_Healthcare/blob/main/Secure_Aggregation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q syft==0.1.29a1


     |████████████████████████████████| 327kB 7.8MB/s 
     |████████████████████████████████| 2.6MB 12.4MB/s 
     |████████████████████████████████| 522kB 44.3MB/s 
     |████████████████████████████████| 1.8MB 43.1MB/s 
     |████████████████████████████████| 2.7MB 49.1MB/s 
     |████████████████████████████████| 676.9MB 25kB/s 
     |████████████████████████████████| 81kB 7.5MB/s 
     |████████████████████████████████| 61kB 6.1MB/s 
     |████████████████████████████████| 61kB 6.6MB/s 
     |████████████████████████████████| 110.5MB 56kB/s 
     |████████████████████████████████| 645kB 41.6MB/s 
     |████████████████████████████████| 61kB 7.4MB/s 
     |████████████████████████████████| 51kB 5.2MB/s 
     |████████████████████████████████| 3.8MB 38.1MB/s 
     |████████████████████████████████| 512kB 42.9MB/s 
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.1.0 which is incompatible.
ERROR: tensorflow 1.15.5 has requirement numpy<1.19.0,>=1.16.0, but

In [2]:
!pip install -q numpy==1.16.4 matplotlib==3.0.3

     |████████████████████████████████| 17.3MB 265kB/s 
     |████████████████████████████████| 13.0MB 189kB/s 
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.1.0 which is incompatible.
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
ERROR: pyarrow 3.0.0 has requirement numpy>=1.16.6, but you'll have numpy 1.16.4 which is incompatible.
ERROR: plotnine 0.6.0 has requirement matplotlib>=3.1.1, but you'll have matplotlib 3.0.3 which is incompatible.
ERROR: mizani 0.6.0 has requirement matplotlib>=3.1.1, but you'll have matplotlib 3.0.3 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: astropy 4.2.1 has requirement numpy>=1.17, but you'll have numpy 1.16.4 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [3]:
!pip install -q torch==1.1.0 torchvision==0.3.0 torchsummary==1.5.1 torchtext==0.3.1

     |████████████████████████████████| 71kB 4.9MB/s 


In [4]:
import torch as th
th.__version__

'1.1.0'

In [5]:
import numpy as np
from torchvision import datasets, transforms
import torchvision.datasets as datasets
from torch.utils.data import Subset
from torch import nn
import torch.nn.functional as F
from torch import optim
import syft as sy
import helper

#Hooking syft to torch
hook = sy.TorchHook(th)

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.7/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.5.so'


In [6]:
#4 different models to pose as 4 different clents
def create_workers():
  workers = []
  cartman = sy.VirtualWorker(hook, id = "rishabh")
  workers.append(rishabh)
  kyle = sy.VirtualWorker(hook, id = "pranav")
  workers.append(pranav)
  kenny = sy.VirtualWorker(hook, id = "divyanshu")
  workers.append(divyanshu)
  stan = sy.VirtualWorker(hook, id = "saumya")
  workers.append(saumya)
  return workers


In [7]:
#to clear out every tensor stored in the list of virtual workers
def clear_workers(workers):
  for worker in workers:
    worker.clear_objects()

In [8]:
#Split datasets to the clients
def create_federated_and_test_loaders(workers, trainset, testset):
  federated_train_loader = sy.FederatedDataLoader(
      trainset.federate(workers), 
      batch_size=32, shuffle=True)

  test_loader = th.utils.data.DataLoader(testset, batch_size=64, shuffle=False)
  return federated_train_loader, test_loader

In [9]:
#Method to bind remote optimizer functions to the Initial states of the remote models 
def create_models(workers, lr):
  remoteModels = list()
  remoteOptimizers = list()
  for worker in workers:
    model = classifier()
    model = model.send(worker)
    remoteOptimizers.append(optim.SGD(model.parameters(), lr))
    remoteModels.append(model)
  return remoteModels, remoteOptimizers
  

In [10]:
#Method to train models on the virtual workers without moving any gradients to the central model until the gradients have been collated.
def create_train_federated_models(workers, loader, lr = 0.12, epoch = 5):
  #sends model to first virtual worker
  virtual_models, virtual_optimizers = create_models(workers, lr)
#   virtual_model = classifier().send(workers[0])
#   optimizer = optim.SGD(virtual_model.parameters(), lr)
  criterion = nn.NLLLoss()
  for n in range(epoch):
    
    #Integer to keep up with first index.
    i = 0
    
    #Integer to keep up with current worker while training
    j = 0
    
    #Integer to count number of mini-batches per worker
    n_mbatch = 0
    
    #Variable to keep up with current worker while looping
    dbLoc = None
    
    #Variable to store cummulative loss.
    cum_loss = 0
    
    
    for batch_idx, (imgs, labels) in enumerate(loader):
      
      #condition to set dbLoc to the first worker
      if i == 0:
        i = 2
        dbLoc = imgs.location
        
      #condition to change dbLoc if img is stored on a different worker and also calculate loss
      if dbLoc is not imgs.location:
        print("The total loss for {0} for epoch {2} is {1}".format(workers[j].id, cum_loss / n_mbatch, n+1))
        dbLoc = imgs.location
        j += 1
        
        #Moving the model to a new worker
#         virtual_model.move(dbLoc)
        
        #Resetting the cummulative loss and batch count to zero for new worker
        cum_loss = 0
        n_mbatch = 0

      virtual_optimizers[j].zero_grad()
      output = virtual_models[j].forward(imgs)
      loss = criterion(output, labels)
      loss.backward()
      virtual_optimizers[j].step()
      cum_loss +=  loss.get().item()
      n_mbatch += 1
    print("The total loss for {0} is {1}".format(workers[j].id, cum_loss / n_mbatch))
  return virtual_models
  
    
  

In [11]:
#to return the model to the central server
def create_central_model(model):
  return model.get()

In [12]:
#Get gradients of client models to their integer representation, encrypt and distribute among the provided workers
def share_gradients(models, workers):
  for model in models:
    model.fc1.weight.data = model.fc1.weight.data.fix_prec().share(*workers)
    model.fc1.bias.data = model.fc1.bias.data.fix_prec().share(*workers)
    model.fc2.weight.data = model.fc2.weight.data.fix_prec().share(*workers)
    model.fc2.bias.data = model.fc2.bias.data.fix_prec().share(*workers)
    model.fc3.weight.data = model.fc3.weight.data.fix_prec().share(*workers)
    model.fc3.bias.data = model.fc3.bias.data.fix_prec().share(*workers)
    model.fc4.weight.data = model.fc4.weight.data.fix_prec().share(*workers)
    model.fc4.bias.data = model.fc4.bias.data.fix_prec().share(*workers)
    model.fc5.weight.data = model.fc5.weight.data.fix_prec().share(*workers)
    model.fc5.bias.data = model.fc5.bias.data.fix_prec().share(*workers)   
  return models
  

In [13]:
#Method to aggregate the gradients accross the shared devices and return the parameters
def aggregate_grads(models):
  fc1_weight = list()
  fc1_bias = list()
  fc2_weight = list()
  fc2_bias = list()
  fc3_weight = list()
  fc3_bias = list()
  fc4_weight = list()
  fc4_bias = list()
  fc5_weight = list()
  fc5_bias = list()
  for model in models:
    fc1_weight.append(model.fc1.weight.data.clone().get())
    fc1_bias.append(model.fc1.bias.data.clone().get())
    fc2_weight.append(model.fc2.weight.data.clone().get())
    fc2_bias.append(model.fc2.bias.data.clone().get())
    fc3_weight.append(model.fc3.weight.data.clone().get())
    fc3_bias.append(model.fc3.bias.data.clone().get())
    fc4_weight.append(model.fc4.weight.data.clone().get())
    fc4_bias.append(model.fc4.bias.data.clone().get())
    fc5_weight.append(model.fc5.weight.data.clone().get())
    fc5_bias.append(model.fc5.bias.data.clone().get())
  params = {}
  params["fc1.weight"] = (sum(fc1_weight) / len(fc1_weight)).get().float_prec()
  params["fc1.bias"] = (sum(fc1_bias) / len(fc1_bias)).get().float_prec()
  params["fc2.weight"] = (sum(fc2_weight) / len(fc2_weight)).get().float_prec()
  params["fc2.bias"] = (sum(fc2_bias) / len(fc2_bias)).get().float_prec()
  params["fc3.weight"] = (sum(fc3_weight) / len(fc3_weight)).get().float_prec()
  params["fc3.bias"] = (sum(fc3_bias) / len(fc3_bias)).get().float_prec()
  params["fc4.weight"] = (sum(fc4_weight) / len(fc4_weight)).get().float_prec()
  params["fc4.bias"] = (sum(fc4_bias) / len(fc4_bias)).get().float_prec()
  params["fc5.weight"] = (sum(fc5_weight) / len(fc5_weight)).get().float_prec()
  params["fc5.bias"] = (sum(fc5_bias) / len(fc5_bias)).get().float_prec()
  return params


In [14]:
#Method to update the central model with the aggregated parameters from the remote models
def update_central_model(params):
  model = classifier()
  model.fc1.weight.data = params["fc1.weight"]
  model.fc1.bias.data = params["fc1.bias"]
  model.fc2.weight.data = params["fc2.weight"]
  model.fc2.bias.data = params["fc2.bias"]
  model.fc3.weight.data = params["fc3.weight"]
  model.fc3.bias.data = params["fc3.bias"]
  model.fc4.weight.data = params["fc4.weight"]
  model.fc4.bias.data = params["fc4.bias"]
  model.fc5.weight.data = params["fc5.weight"]
  model.fc5.bias.data = params["fc5.bias"]  
  return model

In [15]:
#Classifier for creating the models
class classifier(nn.Module):
  def __init__(self):
    super().__init__() 
    self.fc1 = nn.Linear(784, 256)
    self.fc2 = nn.Linear(256, 128)
    self.fc3 = nn.Linear(128, 64)
    self.fc4 = nn.Linear(64, 32)
    self.fc5 = nn.Linear(32, 10)
    
  def forward(self, x):
    x = x.view(x.shape[0], -1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    x = F.relu(self.fc4(x))
    x = F.log_softmax(self.fc5(x), dim = 1)   
    return x
 

In [16]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

0it [00:00, ?it/s]

0it [00:00, ?it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz


FileNotFoundError: ignored

In [ ]:
train = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.5,), (0.5,))
                             ]))
  

test = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.5,), (0.5,))
                             ]))

In [ ]:
from syft import VirtualWorker
workers = create_workers()
clear_workers(workers)

In [ ]:
federatedset, testset= create_federated_and_test_loaders(workers, mnist_trainset, mnist_testset)

In [ ]:
remoteModels = create_train_federated_models(workers, federatedset, lr = 0.15, epoch = 20)

In [ ]:
remoteModels = create_train_federated_models(workers, federatedset, lr = 0.15, epoch = 20)

In [ ]:
params  = aggregate_grads(remoteModels)

In [ ]:
central_model = update_central_model(params)
